# Table des Matières
1. [Introduction et Sélection des données](#intro)
2. [Exploration et traitement des données](#traitement)
3. [Modélisation et évaluation](#model)
4. [Comunication des résultats](#comm)
5. [Retour d'expérience](#retour) 

  
 

---

## Introduction et sélection des données <a class="anchor" id="intro"></a>

Dans le cadre de notre projet de machine Learning, nous allons traiter un sujet d'apprentissage supervisé à partir de données ouvertes. Tout au long de notre projet, nous utiliserons l'IA générative (ChatGPT) pour nous aider dans notre travail. 

Dans un premier temps, nous commençons par demander à ChatGPT des idées de sujets.

Jeu de données ML généré par l'IA 
- DPE (Analyse énergétique, classification des logements)
- Transports en commun (prédiction affluence, optimisation trajet)
- Pollution (prédiction des pics de pollution, classification zones polluées)
- Reconnaissance d'image
- Traitement du langage naturel
- Donnée de santé
- Donnée financière économique (prédiction de tendance..)

Analyse critique de l'IA
L'IA propose souvent des sujets très classiques, déjà réalisé. Il propose même des projets déjà tout fait sur Kaggle. Il manque d'originalité ! 

### Données initiales <a class="anchor" id="intro"></a>


**Changement de Problématique : Explication**

Initialement de notre projet portait sur la prédiction des passoires thermiques à partir des données DPE : **Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?** 

Cependant, après une analyse approfondie des données, nous avons décidé de changer de jeu de données et de problématique.

La problématique initiale, s'est révélée trop simple pour un sujet de machine learning. Les données DPE sont directement corrélées avec les passoires thermiques, ce qui limite les défis en termes de modélisation et d'analyse.

Nous avons donc opté pour une nouvelle problématique, avec un jeu de données moins binaires. 


**Problématique générée avec L'IA sur les données DPE**

- Peut-on prédire la consommation énergétique d’un logement à partir de ses caractéristiques issues du DPE ?
- Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?
- Peut-on prédire la classe énergétique (A à G) d’un logement à partir des caractéristiques fournies par le DPE ?

**Critique de l'IA sur notre approche précedente**

Votre approche est bien structurée et inclut des variables clés dans des domaines pertinents pour prédire si un logement est une passoire thermique. 

Points positifs :
Les variables sélectionnées correspondent bien à des déterminants majeurs de la performance énergétique.
Vous incluez à la fois des variables quantitatives (ex. : consommation énergétique, déperditions) et qualitatives (ex. : type d'installation de chauffage, zone climatique), ce qui permet une modélisation riche.

Critiques et suggestions :
Les bases de données DPE peuvent contenir des valeurs manquantes ou aberrantes (ex. : surfaces non renseignées, consommations incohérentes).
Vérifiez les distributions des variables pour détecter des anomalies.
Plusieurs variables peuvent être fortement corrélées. Une analyse des corrélations peut être utile.
Les passoires thermiques (F et G) pourraient représenter une minorité des données. Cela peut biaiser l’entraînement du modèle.
Les bâtiments anciens sont souvent surreprésentés parmi les passoires thermiques, ce qui peut conduire le modèle à négliger d'autres facteurs. Une analyse exploratoire approfondie est nécessaire.
Les algorithmes robustes pour ce type de problème incluent :
 Forêts aléatoires (Random Forests): pour leur interprétabilité et leur capacité à gérer des variables mixtes.
 Gradient Boosting (XGBoost, LightGBM) : pour leur performance sur des ensembles déséquilibrés.
 Réseaux de neurones : si les données sont enrichies avec des caractéristiques complexes (géographiques, temporelles).

lien données DPE : https://drive.google.com/file/d/1nUbA6m3SQ9PXpc9i9tD6yADe0fDFYKTc/view?usp=drive_link


### Données définitives <a class="anchor" id="intro"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df_flight = pd.read_csv("data/itineraries_sample.csv")

Afin de répondre à notre problématique, nous utilisons un jeu de données issu de vols aller simple du site Expedia entre le 16 avril 2022 et le 5 octobre 2022. 

Voici l'intégralité des variables présentes dans notre jeu de données : 

- **Identifiants et Dates**  
  - `legId` : Identifiant unique pour chaque vol.  
  - `searchDate` : Date de la recherche effectuée sur Expedia (AAAA-MM-JJ).  
  - `flightDate` : Date du vol (AAAA-MM-JJ).  

- **Informations sur les Aéroports**  
  - `startingAirport` : Code IATA (trois caractères) de l’aéroport de départ.  
  - `destinationAirport` : Code IATA (trois caractères) de l’aéroport d’arrivée.  

- **Tarification et Conditions**  
  - `fareBasisCode` : Code de base tarifaire.  
  - `baseFare` : Tarif de base du billet (en USD).  
  - `totalFare` : Tarif total incluant taxes et frais (en USD).  
  - `isBasicEconomy` : Indique si le billet appartient à la classe économique basique (booléen).  
  - `isRefundable` : Indique si le billet est remboursable (booléen).  

- **Caractéristiques du Vol**  
  - `isNonStop` : Indique si le vol est direct (booléen).  
  - `travelDuration` : Durée totale du trajet (heures et minutes).  
  - `elapsedDays` : Nombre de jours écoulés avant le vol.  
  - `seatsRemaining` : Nombre de sièges restants disponibles.  
  - `totalTravelDistance` : Distance totale parcourue en miles.  

- **Segments du Vol**  
Les données contiennent également des informations détaillées sur chaque segment du trajet :  
  - Heures de départ et d’arrivée (`segmentsDepartureTimeRaw`, `segmentsArrivalTimeRaw`).  
  - Codes aéroportuaires des départs et arrivées (`segmentsDepartureAirportCode`, `segmentsArrivalAirportCode`).  
  - Compagnies aériennes et avions utilisés (`segmentsAirlineName`, `segmentsEquipmentDescription`).  
  - Durée et distance de chaque segment (`segmentsDurationInSeconds`, `segmentsDistance`).  
  - Cabine (`segmentsCabinCode`).  


In [3]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000373 entries, 0 to 1000372
Data columns (total 27 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   legId                              1000373 non-null  object 
 1   searchDate                         1000373 non-null  object 
 2   flightDate                         1000373 non-null  object 
 3   startingAirport                    1000373 non-null  object 
 4   destinationAirport                 1000373 non-null  object 
 5   fareBasisCode                      1000373 non-null  object 
 6   travelDuration                     1000373 non-null  object 
 7   elapsedDays                        1000373 non-null  int64  
 8   isBasicEconomy                     1000373 non-null  bool   
 9   isRefundable                       1000373 non-null  bool   
 10  isNonStop                          1000373 non-null  bool   
 11  baseFare                

## Exploration et traitement des données <a class="anchor" id="traitement"></a>

### Traitement des valeurs manquantes <a class="anchor" id="traitement"></a>

In [4]:
pourcentage_valeurs_manquantes = df_flight.isnull().mean() * 100
pourcentage_valeurs_manquantes

legId                                0.000000
searchDate                           0.000000
flightDate                           0.000000
startingAirport                      0.000000
destinationAirport                   0.000000
fareBasisCode                        0.000000
travelDuration                       0.000000
elapsedDays                          0.000000
isBasicEconomy                       0.000000
isRefundable                         0.000000
isNonStop                            0.000000
baseFare                             0.000000
totalFare                            0.000000
seatsRemaining                       0.000000
totalTravelDistance                  7.416434
segmentsDepartureTimeEpochSeconds    0.000000
segmentsDepartureTimeRaw             0.000000
segmentsArrivalTimeEpochSeconds      0.000000
segmentsArrivalTimeRaw               0.000000
segmentsArrivalAirportCode           0.000000
segmentsDepartureAirportCode         0.000000
segmentsAirlineName               

In [5]:
# Retirer toutes les lignes contenant des valeurs manquantes
df_cleaned = df_flight.dropna()

# Vérifier que les valeurs manquantes ont été supprimées
print(df_cleaned.isnull().sum())

legId                                0
searchDate                           0
flightDate                           0
startingAirport                      0
destinationAirport                   0
fareBasisCode                        0
travelDuration                       0
elapsedDays                          0
isBasicEconomy                       0
isRefundable                         0
isNonStop                            0
baseFare                             0
totalFare                            0
seatsRemaining                       0
totalTravelDistance                  0
segmentsDepartureTimeEpochSeconds    0
segmentsDepartureTimeRaw             0
segmentsArrivalTimeEpochSeconds      0
segmentsArrivalTimeRaw               0
segmentsArrivalAirportCode           0
segmentsDepartureAirportCode         0
segmentsAirlineName                  0
segmentsAirlineCode                  0
segmentsEquipmentDescription         0
segmentsDurationInSeconds            0
segmentsDistance         

**Sélection des données** 

Dans un premier temps, nous sélectionnons les variables qui nous semblent pertinantes. 

Voici la liste des variables que nous retenons : 
- `legId`
- `searchDate`
- `flightDate`
- `startingAirport`
- `destinationAirport`
- `fareBasisCode`
- `travelDuration`
- `elapsedDays`
- `isBasicEconomy`
- `isRefundable`
- `isNonStop`
- `totalFare`
- `seatsRemaining`
- `totalTravelDistance`


In [6]:
var_select = [
'legId',
'searchDate',
'flightDate',
'startingAirport',
'destinationAirport',
'fareBasisCode',
'travelDuration',
'elapsedDays',
'isBasicEconomy',
'isRefundable',
'isNonStop',
'totalFare',
'seatsRemaining',
'totalTravelDistance'
]

In [7]:
df_cleaned = df_cleaned[var_select]
df_cleaned

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance
1,6203bbd77fbd8e40021ee3e88ffa9edc,2022-04-16,2022-04-17,ATL,IAD,KA0NA0MC,PT12H22M,0,False,False,False,746.14,2,1224.0
2,34bb71c85bd77485193f5d83c553d783,2022-04-16,2022-04-17,ATL,JFK,EAA0OKEN,PT9H15M,0,False,False,False,672.19,1,762.0
3,691ae27539fcaab7ea209c67d36a6bdb,2022-04-16,2022-04-17,ATL,LAX,V0AGZNN1,PT7H47M,0,False,False,False,370.60,4,1954.0
4,5acd2ad9bbc5dfcdbfb2d70920c45a55,2022-04-16,2022-04-17,ATL,LAX,MA0QA0MQ,PT4H57M,0,False,False,True,598.61,7,1943.0
5,48250c83294e10c36c992ec208fc62f7,2022-04-16,2022-04-17,ATL,LGA,KA0NX0MQ,PT2H12M,0,False,False,True,244.60,1,762.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000368,9877500f9069b28d3bbde94206bb48c0,2022-10-05,2022-11-12,DFW,MIA,VUAWZNN1,PT6H55M,0,False,False,False,244.60,7,2100.0
1000369,75f5c5422edf8d0074b624106170fc2f,2022-10-05,2022-11-12,DFW,OAK,GH4OAJBN,PT7H42M,0,True,False,False,152.60,7,2335.0
1000370,b48181202b764fa7fddbf23823fa2903,2022-10-05,2022-11-12,DFW,PHL,TAVOA0BG,PT6H29M,0,True,False,False,158.60,9,1392.0
1000371,5ed2af184418e2a2f67d2839a593c1e9,2022-10-05,2022-11-12,DFW,PHL,QUAIXSM3,PT9H49M,1,False,False,False,463.20,7,1304.0


### Corrélation entre les variables <a class="anchor" id="traitement"></a>

### Répartion des variables <a class="anchor" id="traitement"></a>

Modèle

In [11]:
import pandas as pd
import numpy as np
import re

# ===============================
# 0) Définition d'une fonction
#    pour parser la durée ISO 8601
# ===============================
def parse_iso8601_duration(duration_str):
    """
    Convertit une durée au format ISO 8601 (ex: 'PT4H57M') 
    en un nombre entier de minutes.
    Si la chaîne est invalide ou NaN, renvoie np.nan.
    """
    if pd.isnull(duration_str):
        return np.nan
    
    # Pattern simplifié pour extraire heures (H) et minutes (M), ex: PT4H57M
    pattern = r'^PT(?:(\d+)H)?(?:(\d+)M)?$'
    match = re.match(pattern, duration_str.strip())
    if not match:
        return np.nan  # Format non conforme ou autre
    
    hours_str, minutes_str = match.groups()
    hours = int(hours_str) if hours_str else 0
    minutes = int(minutes_str) if minutes_str else 0
    
    total_minutes = hours * 60 + minutes
    return total_minutes

# ===============================
# 1) Conversion des dates
# ===============================
df_cleaned['searchDate'] = pd.to_datetime(df_cleaned['searchDate'], errors='coerce')
df_cleaned['flightDate'] = pd.to_datetime(df_cleaned['flightDate'], errors='coerce')

# ===============================
# 2) daysBetweenSearchAndFlight
#    (flightDate - searchDate) en jours
# ===============================
df_cleaned['daysBetweenSearchAndFlight'] = (df_cleaned['flightDate'] - df_cleaned['searchDate']).dt.days

# ===============================
# 3) searchDayOfWeek et flightDayOfWeek
#    (0 = Lundi, ..., 6 = Dimanche)
# ===============================
df_cleaned['searchDayOfWeek'] = df_cleaned['searchDate'].dt.dayofweek
df_cleaned['flightDayOfWeek'] = df_cleaned['flightDate'].dt.dayofweek

# ===============================
# 4) searchMonth, flightMonth, flightYear
# ===============================
df_cleaned['searchMonth'] = df_cleaned['searchDate'].dt.month
df_cleaned['flightMonth'] = df_cleaned['flightDate'].dt.month
df_cleaned['flightYear'] = df_cleaned['flightDate'].dt.year

# ===============================
# 5) flightWeekend (booléen)
#    True si Samedi ou Dimanche
# ===============================
df_cleaned['flightWeekend'] = df_cleaned['flightDayOfWeek'].isin([5, 6])

# ===============================
# 6) Conversion de travelDuration
#    (ex: 'PT4H57M' -> minutes int)
# ===============================
df_cleaned['travelDurationMinutes'] = df_cleaned['travelDuration'].apply(parse_iso8601_duration)

# ===============================
# 7) fareType
#    Basé sur la 1ère lettre de fareBasisCode
# ===============================
FIRST = {'F', 'A', 'P'}
BUSINESS = {'C', 'J', 'D', 'I', 'Z'}
PREMIUM_ECONOMY = {'W', 'S', 'R'}
ECONOMY = {'Y', 'B', 'M', 'U', 'H', 'Q', 'K', 'L', 'G', 'V', 'T', 'N', 'X', 'O', 'E'}

def map_fare_type(code):
    """
    Retourne la cabine probable (First, Business, Premium Economy, Economy, Unknown)
    en se basant sur la première lettre du fareBasisCode.
    """
    if pd.isnull(code) or not code:
        return 'Unknown'
    
    first_char = str(code).strip().upper()[0]
    
    if first_char in FIRST:
        return 'First'
    elif first_char in BUSINESS:
        return 'Business'
    elif first_char in PREMIUM_ECONOMY:
        return 'Premium Economy'
    elif first_char in ECONOMY:
        return 'Economy'
    else:
        return 'Unknown'

df_cleaned['fareType'] = df_cleaned['fareBasisCode'].apply(map_fare_type)

# ===============================
# 8) isOvernightFlight
#    Si elapsedDays > 0
# ===============================
df_cleaned['isOvernightFlight'] = df_cleaned['elapsedDays'] > 0

# ===============================
# 9) distanceBands
#    short-haul (<500), medium-haul (<=1500), long-haul (>1500)
# ===============================
def define_distance_band(x):
    if pd.isnull(x):
        return 'unknown'
    elif x < 500:
        return 'short-haul'
    elif x <= 1500:
        return 'medium-haul'
    else:
        return 'long-haul'

df_cleaned['distanceBands'] = df_cleaned['totalTravelDistance'].apply(define_distance_band)

# ===============================
# 10) Constitution de df_final
#     Avec les features finales + la cible
# ===============================
df_final = df_cleaned[[
    'daysBetweenSearchAndFlight', 
    'startingAirport',
    'destinationAirport',
    'searchMonth', 
    'searchDayOfWeek',
    'flightYear', 
    'flightMonth', 
    'flightDayOfWeek', 
    'flightWeekend', 
    'fareType',
    'elapsedDays', 
    'isOvernightFlight',
    'totalTravelDistance', 
    'distanceBands',
    'travelDurationMinutes',      # La nouvelle colonne
    'totalFare'
]]

# Vérification
df_final.head()

,daysBetweenSearchAndFlight,startingAirport,destinationAirport,searchMonth,searchDayOfWeek,flightYear,flightMonth,flightDayOfWeek,flightWeekend,fareType,elapsedDays,isOvernightFlight,totalTravelDistance,distanceBands,travelDurationMinutes,totalFare
1,1,ATL,IAD,4,5,2022,4,6,True,Economy,0,False,1224.0,medium-haul,742.0,746.14
2,1,ATL,JFK,4,5,2022,4,6,True,Economy,0,False,762.0,medium-haul,555.0,672.19
3,1,ATL,LAX,4,5,2022,4,6,True,Economy,0,False,1954.0,long-haul,467.0,370.60
4,1,ATL,LAX,4,5,2022,4,6,True,Economy,0,False,1943.0,long-haul,297.0,598.61
5,1,ATL,LGA,4,5,2022,4,6,True,Economy,0,False,762.0,medium-haul,132.0,244.60


In [12]:
import pandas as pd
import numpy as np

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modèles
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Métriques
from sklearn.metrics import mean_squared_error, r2_score

# =====================
# 1) Définition de la cible (y) et des features (X)
target = 'totalFare'

features = [
    'daysBetweenSearchAndFlight',
    'startingAirport',
    'destinationAirport',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'flightWeekend',
    'fareType',
    'elapsedDays',
    'isOvernightFlight',
    'totalTravelDistance',
    'distanceBands',
    'travelDurationMinutes'
]

# On suppose que df_final contient déjà ces colonnes
X = df_final[features].copy()
y = df_final[target].copy()

# =====================
# 2) Séparation en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =====================
# 3) Gestion des booléens (optionnel)
#    Si tu préfères traiter flightWeekend, isOvernightFlight en 0/1 (numérique):
bool_cols = ['flightWeekend', 'isOvernightFlight']
for col in bool_cols:
    X_train[col] = X_train[col].astype(int, errors="ignore")
    X_test[col] = X_test[col].astype(int, errors="ignore")

# =====================
# 4) Identification des colonnes numériques et catégorielles
#    Selon ton choix, on traite flightWeekend, isOvernightFlight comme numériques.
numeric_features = [
    'daysBetweenSearchAndFlight',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'elapsedDays',
    'flightWeekend',
    'isOvernightFlight',
    'totalTravelDistance',
    'travelDurationMinutes'
]

# Pour les colonnes purement catégorielles
categorical_features = [
    'fareType',
    'distanceBands',
    'startingAirport',
    'destinationAirport'
]

# =====================
# 5) Construction du préprocesseur avec ColumnTransformer
from sklearn.compose import make_column_selector

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),   # Remplacement des NaN
    ("scaler", StandardScaler())                   # Mise à l'échelle standard
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),  
    ("onehot", OneHotEncoder(handle_unknown="ignore"))                     
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# =====================
# 6) Définition des modèles à tester
models = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

# =====================
# 7) Entraînement et évaluation de chaque modèle
results = []

for model_name, model in models.items():
    # Pipeline = Preprocessing + Modèle
    pipe = Pipeline([
        ("preprocessing", preprocessor),
        ("regressor", model)
    ])
    
    # Entraînement
    pipe.fit(X_train, y_train)
    
    # Prédiction sur le test set
    y_pred = pipe.predict(X_test)
    
    # Évaluation
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "model": model_name,
        "rmse": rmse,
        "r2": r2
    })

# =====================
# 8) Affichage des résultats
results_df = pd.DataFrame(results)
print(results_df)

              model        rmse        r2
0      DecisionTree  150.239420  0.417621
1      RandomForest  106.998941  0.704609
2  LinearRegression  151.092062  0.410991
3           XGBoost  117.837593  0.641734


In [13]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modèles
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Métriques
from sklearn.metrics import mean_squared_error, r2_score

###############################################################################
# 1) Définition de la cible et des features
###############################################################################
target = 'totalFare'

# Ton jeu de features, y compris les aéroports et la duration en minutes
features = [
    'daysBetweenSearchAndFlight',
    'startingAirport',
    'destinationAirport',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'flightWeekend',
    'fareType',
    'elapsedDays',
    'isOvernightFlight',
    'totalTravelDistance',
    'distanceBands',
    'travelDurationMinutes'
]

# On suppose que df_final est déjà disponible et contient ces colonnes
X = df_final[features].copy()
y = df_final[target].copy()

# Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

###############################################################################
# 2) Conversion des booléens (flightWeekend, isOvernightFlight) en int (0/1)
###############################################################################
bool_cols = ['flightWeekend', 'isOvernightFlight']
for col in bool_cols:
    X_train[col] = X_train[col].astype(int, errors="ignore")
    X_test[col] = X_test[col].astype(int, errors="ignore")

###############################################################################
# 3) Distinction colonnes numériques vs. colonnes catégorielles
###############################################################################
numeric_features = [
    'daysBetweenSearchAndFlight',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'elapsedDays',
    'flightWeekend',
    'isOvernightFlight',
    'totalTravelDistance',
    'travelDurationMinutes'
]

categorical_features = [
    'fareType',
    'distanceBands',
    'startingAirport',
    'destinationAirport'
]

# Prétraitement : imputation + standardisation pour numeric, imputation + OneHot pour categorical
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

###############################################################################
# 4) GridSearchCV pour RandomForest
###############################################################################
pipe_rf = Pipeline([
    ("preprocessing", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

# Exemple d'hyperparamètres (à ajuster selon tes besoins)
param_grid_rf = {
    "regressor__n_estimators": [50, 100],
    "regressor__max_depth": [None, 10, 20],
    "regressor__min_samples_leaf": [1, 5]
}

grid_search_rf = GridSearchCV(
    estimator=pipe_rf,
    param_grid=param_grid_rf,
    cv=3,  # 3-fold cross validation
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Entraînement
grid_search_rf.fit(X_train, y_train)

print("========== RandomForest GridSearch ==========")
print("Best params (RF):", grid_search_rf.best_params_)
print("Best CV score (RF - RMSE):", -grid_search_rf.best_score_)

# Évaluation sur le test set
best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
print(f"[RF] Test set RMSE: {rmse_rf:.2f}, R²: {r2_rf:.3f}")

###############################################################################
# 5) GridSearchCV pour XGBoost (optionnel)
###############################################################################
pipe_xgb = Pipeline([
    ("preprocessing", preprocessor),
    ("regressor", XGBRegressor(random_state=42, verbosity=0))
])

param_grid_xgb = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 6, 10],
    "regressor__learning_rate": [0.01, 0.1],
    "regressor__subsample": [0.8, 1.0]
}

grid_search_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=param_grid_xgb,
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb.fit(X_train, y_train)

print("\n========== XGBoost GridSearch ==========")
print("Best params (XGB):", grid_search_xgb.best_params_)
print("Best CV score (XGB - RMSE):", -grid_search_xgb.best_score_)

best_xgb_model = grid_search_xgb.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"[XGB] Test set RMSE: {rmse_xgb:.2f}, R²: {r2_xgb:.3f}")

###############################################################################
# 6) Conclusion
###############################################################################
# Tu obtiens:
# - Meilleurs hyperparamètres pour RandomForest & XGBoost
# - Leurs performances RMSE / R² sur le set de test
###############################################################################

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


========== RandomForest GridSearch ==========
Best params (RF): {'regressor__max_depth': None, 'regressor__min_samples_leaf': 1, 'regressor__n_estimators': 100}
Best CV score (RF - RMSE): 112.66567744123013
[RF] Test set RMSE: 107.00, R²: 0.705
Fitting 3 folds for each of 24 candidates, totalling 72 fits

========== XGBoost GridSearch ==========
Best params (XGB): {'regressor__learning_rate': 0.1, 'regressor__max_depth': 10, 'regressor__n_estimators': 200, 'regressor__subsample': 0.8}
Best CV score (XGB - RMSE): 110.94292108539868
[XGB] Test set RMSE: 106.84, R²: 0.705
